# Global

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_from_gdrive = '05' # put '05' for the 05 class and '06' for the 06 class

# Import

In [ ]:
!pip3 install nltk==3.8.1
!pip install gdown
!pip install spacy
!python -m spacy download en_core_web_sm

!pip install sentence-transformers -U

2024-01-08 16:38:09.315625: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 16:38:09.315723: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 16:38:09.318568: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 16:38:09.328890: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-08 16:38:10.764702: W tensorflow/compiler/tf2

In [ ]:
import pandas as pd
from tqdm import tqdm
import nltk
import gdown
import spacy
import re
from nltk.corpus import stopwords
from collections import defaultdict
import numpy as np
import csv

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

# Import the answers

In [ ]:
if file_from_gdrive == '05':
  file_id = '1pTjO1V0bVt4H7NNvPnngib2CsUb8DJ9W'

elif file_from_gdrive == '06':
  file_id = '1PQG87AmmIYqczZJx3RYlm6YNLOGeEnAo'

download_link = f'https://drive.google.com/uc?id={file_id}'
content = gdown.download(download_link, quiet=False)


# Define the column names for the DataFrame
columns = ['word', 'question', 'answer']

# Read the text file into a DataFrame
df = pd.read_csv(content, sep='\t', header=None, names=columns)

# Display the DataFrame
df

Downloading...
From: https://drive.google.com/uc?id=1pTjO1V0bVt4H7NNvPnngib2CsUb8DJ9W
To: /content/class_05.clean.txt
100%|██████████| 503k/503k [00:00<00:00, 72.5MB/s]


,word,question,answer
0,lion,Is he a carnivore or herbivore?,Lion is a carnivore.
1,lion,Is it a vertebrate or invertebrate?,Lion is a vertebrate.
2,lion,Has it ever been domesticated?,Lions have been trained to some extent in capt...
3,lion,Is it a mammal?,"Yes, the lion is a mammal."
4,lion,Does it have fur?,"Yes, lions have fur."
...,...,...,...
4087,quokka,What are its dimensions?,Quokkas are small marsupials with a body lengt...
4088,quokka,Does the animal have moist or scaly skin?,"Quokkas have fur-covered skin, and their skin ..."
4089,quokka,Can the animal echolocate?,Quokkas do not echolocate. They rely on visual...
4090,quokka,Can the animal metamorphose?,"No, quokkas do not undergo metamorphosis. They..."


# Preprocess

## Remove stopwords and tokenize

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.remove('no') # to remove 'no' from the stopwords, otherwise it cannot be a keyword successively
# stop_words.remove('not') # to remove 'not' from the stopwords, otherwise it cannot be a keyword successively
stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 're',
 's

In [ ]:
nlp = spacy.load("en_core_web_sm")

def preprocess_text(sentence, stopwords, lemmatize=True):
  doc = nlp(sentence)
  sentence_tokens = []
  for token in doc:

    if token.pos_ not in ['NOUN', 'ADJ', 'VERB'] and token.text.lower() not in {'yes', 'no'}:
      continue

    token_text = token.lemma_ if lemmatize else token.text
    token_text = token_text.lower()

    if token_text in stopwords:
      continue
    sentence_tokens.append(token_text)
  return sentence_tokens

In [ ]:
# remove common words and tokenize
prep_answs = [preprocess_text(sentence, stop_words, lemmatize=True) for sentence in tqdm(df['answer'])]

100%|██████████| 4092/4092 [01:02<00:00, 65.06it/s] 


In [ ]:
prep_answs

[['carnivore'],
 ['vertebrate'],
 ['lion', 'train', 'extent', 'captivity', 'domesticate', 'dog', 'cat'],
 ['yes', 'lion', 'mammal'],
 ['yes', 'lion', 'fur'],
 ['lion', 'find', 'grassland', 'savanna', 'open', 'woodland'],
 ['lion', 'carnivore', 'eat', 'meat', 'hunt', 'group'],
 ['lion', 'leg'],
 ['no', 'lion', 'give', 'birth', 'live', 'young'],
 ['lion', 'predator'],
 ['lion', 'leg', 'no', 'wing', 'fin'],
 ['lion', 'find', 'small', 'population'],
 ['wild', 'lion', 'dangerous', 'human', 'close', 'encounter'],
 ['lion', 'eat', 'human'],
 ['yes', 'lion', 'know', 'powerful', 'agile', 'jump'],
 ['lion', 'short', 'stand', 'human', 'body', 'length'],
 ['lion',
  'live',
  'social',
  'group',
  'call',
  'pride',
  'consist',
  'relate',
  'female',
  'offspring'],
 ['lion', 'suitable', 'legal', 'keep', 'pet', 'wild', 'nature'],
 ['lion',
  'migrate',
  'long',
  'distance',
  'movement',
  'influence',
  'factor',
  'food',
  'availability',
  'water',
  'source'],
 ['lion', 'intelligent', 'p

## Preprocessed answers substitution

In [ ]:
# add the column 'prep_answer' to the dataframe and populate it
df['prep_answer'] = [0 for i in range(len(df))]

for i in range (len(df)):
  df['prep_answer'][i] = prep_answs[i]

<ipython-input-11-b64b0a520239>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prep_answer'][i] = prep_answs[i]


In [ ]:
df

,word,question,answer,prep_answer
0,lion,Is he a carnivore or herbivore?,Lion is a carnivore.,[carnivore]
1,lion,Is it a vertebrate or invertebrate?,Lion is a vertebrate.,[vertebrate]
2,lion,Has it ever been domesticated?,Lions have been trained to some extent in capt...,"[lion, train, extent, captivity, domesticate, ..."
3,lion,Is it a mammal?,"Yes, the lion is a mammal.","[yes, lion, mammal]"
4,lion,Does it have fur?,"Yes, lions have fur.","[yes, lion, fur]"
...,...,...,...,...
4087,quokka,What are its dimensions?,Quokkas are small marsupials with a body lengt...,"[small, marsupial, body, length, cm, inch, tai..."
4088,quokka,Does the animal have moist or scaly skin?,"Quokkas have fur-covered skin, and their skin ...","[fur, cover, skin, skin, moist]"
4089,quokka,Can the animal echolocate?,Quokkas do not echolocate. They rely on visual...,"[echolocate, rely, visual, vocal, communication]"
4090,quokka,Can the animal metamorphose?,"No, quokkas do not undergo metamorphosis. They...","[no, undergo, metamorphosis, give, birth, live..."


## Check the number of questions for each concept

In [ ]:
num_words = 0
for i in range(len(df['word'])):
  if i == 0:
    num_words += 1
  elif df['word'][i] != df['word'][i-1]:
    num_words += 1

In [ ]:
num_questions = int(len(df) / num_words)

In [ ]:
print('Number of questions for each concept: ', num_questions)

Number of questions for each concept:  44


# Counting
Here we create a dictionary for each question in which there are the (token:frequency) couples that are in turn in all the answers for that question.

For instance:<br>
question 1: .....? <br>
answer for concept 1: ......<br>
.<br>
.<br>
.<br>
answer for concept n: ......<br><br>


In this case, we have a dictionary for question 1 with the keywords extrapolated by all its answers (i.e. answer 1, ..., answer n).



In [ ]:
# create a dict for each question (i.e. num_questions dicts), each one with (token:frequency) that are found in that question

dict_list = []

for i in tqdm(range(num_questions)):
    frequency = defaultdict(int)

    for j in range(i, len(df), num_questions):
        for token in df['prep_answer'][j]:
            frequency[token] += 1

            if token in {'yes', 'no'}:
               frequency[token] += 1000000 # this is to enforce the keyword to be 'yes', 'no'

    dict_list.append(frequency)

100%|██████████| 44/44 [00:00<00:00, 1049.26it/s]


In [ ]:
len(dict_list) # num_questions is right

44

In [ ]:
# print a dictionary to check if all is ok
dict_list[8]

defaultdict(int,
            {'no': 50000050,
             'lion': 1,
             'give': 55,
             'birth': 55,
             'live': 55,
             'young': 53,
             'tiger': 1,
             'elephant': 2,
             'giraffe': 1,
             'zebra': 1,
             'cheetah': 1,
             'gorilla': 1,
             'kangaroo': 1,
             'koala': 1,
             'panda': 2,
             'polar': 1,
             'bear': 3,
             'grizzly': 1,
             'brown': 1,
             'yes': 42000042,
             'penguin': 1,
             'lay': 26,
             'egg': 57,
             'dolphin': 1,
             'whale': 1,
             'shark': 1,
             'reproduce': 14,
             'oviparous': 1,
             'viviparous': 1,
             'depend': 1,
             'specie': 3,
             'octopus': 1,
             'female': 12,
             'guard': 2,
             'care': 1,
             'hatch': 6,
             'jellyfish': 1,
          

# Keywords selection
To select the keywords for each concept and each question, we take the dictionary corresponding to that question, filter it by keeping only the keywords of the corresponding answer and:<br>
if there is only one keyword, then keep it;<br>
else keep the keyword with the highest frequency;

In [ ]:
keywords = []

j = 0

for i in tqdm(range(len(df))):
  keys_to_consider = df['prep_answer'][i] # to consider only the keys in the answer
  filtered_dict = {key: value for key, value in dict_list[j].items() if key in keys_to_consider} # filtered version of the dict with only the considered keys

  max_key = max(filtered_dict, key=lambda k: filtered_dict[k]) # select the key corresponding to the max frequency (i.e. the keyword)
  keywords.append(max_key)

  j += 1
  if j > num_questions - 1:
    j = 0


100%|██████████| 4092/4092 [00:00<00:00, 17770.16it/s]


In [ ]:
len(keywords) # len(df) is right

4092

## Add the column 'keyword' to the dataframe (for a better view)

In [ ]:
df['keyword'] = keywords

In [ ]:
df

,word,question,answer,prep_answer,keyword
0,lion,Is he a carnivore or herbivore?,Lion is a carnivore.,[carnivore],carnivore
1,lion,Is it a vertebrate or invertebrate?,Lion is a vertebrate.,[vertebrate],vertebrate
2,lion,Has it ever been domesticated?,Lions have been trained to some extent in capt...,"[lion, train, extent, captivity, domesticate, ...",domesticate
3,lion,Is it a mammal?,"Yes, the lion is a mammal.","[yes, lion, mammal]",yes
4,lion,Does it have fur?,"Yes, lions have fur.","[yes, lion, fur]",yes
...,...,...,...,...,...
4087,quokka,What are its dimensions?,Quokkas are small marsupials with a body lengt...,"[small, marsupial, body, length, cm, inch, tai...",inch
4088,quokka,Does the animal have moist or scaly skin?,"Quokkas have fur-covered skin, and their skin ...","[fur, cover, skin, skin, moist]",skin
4089,quokka,Can the animal echolocate?,Quokkas do not echolocate. They rely on visual...,"[echolocate, rely, visual, vocal, communication]",echolocate
4090,quokka,Can the animal metamorphose?,"No, quokkas do not undergo metamorphosis. They...","[no, undergo, metamorphosis, give, birth, live...",no


## Write the output into a tsv file

In [ ]:
df_tsv = df.drop(columns=['answer', 'prep_answer'])

In [ ]:
df_tsv

,word,question,keyword
0,lion,Is he a carnivore or herbivore?,carnivore
1,lion,Is it a vertebrate or invertebrate?,vertebrate
2,lion,Has it ever been domesticated?,domesticate
3,lion,Is it a mammal?,yes
4,lion,Does it have fur?,yes
...,...,...,...
4087,quokka,What are its dimensions?,inch
4088,quokka,Does the animal have moist or scaly skin?,skin
4089,quokka,Can the animal echolocate?,echolocate
4090,quokka,Can the animal metamorphose?,no


In [ ]:
file_path = '/content/drive/MyDrive/Hot_topics_in_nlp/homework_2/part_2/class_' + file_from_gdrive + '.values.tsv'

# Write DataFrame to a TSV file
df_tsv.to_csv(file_path, sep='\t', index=False)

# PPMI

 $$
 PPMI(w_1, w_2) =
 \begin{cases}
 & PMI(w_1, w_2) \hspace{10mm} if \: PMI(w_1, w_2) > 0 \\
 & 0 \hspace{28mm} otherwise \\
 \end{cases}
$$

Define the co-occurence matrix function

In [ ]:
def co_occurrence(sentences, window_size):
    d = defaultdict(int)
    vocab = set()
    for words in tqdm(sentences):
        for i in range(len(words)):
            word = words[i]
            vocab.add(word)
            next_word = words[i+1 : i+1+window_size]
            for t in next_word:
                key = tuple( sorted([t, word]) )
                d[key] += 1

    # transform the dictionary into dataframe
    vocab = sorted(vocab)
    df = pd.DataFrame(data=np.zeros((len(vocab), len(vocab)), dtype=np.int16),
                      index=vocab,
                      columns=vocab)
    for key, value in tqdm(d.items()):
        df.at[key[0], key[1]] = value
        df.at[key[1], key[0]] = value
    return df

To make an example, compute the co-occurence (and consequently the ppmi matrix) for the first question. So, we take the answer to the first question for all the concepts

In [ ]:
# compute the co-occurence  matrix for the keywords of each question
datafr = co_occurrence([list(dict_list[0].keys())], 5) # co-occurence matrix for the keywords of the first question
datafr

100%|██████████| 830/830 [00:00<00:00, 23550.28it/s]


,algae,alligator,amount,amphibian,animal,ant,anteater,aphid,aquatic,baboon,...,turtle,variety,vegetable,vegetation,walrus,warthog,whale,wombat,worm,zebra
algae,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
alligator,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
amount,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
amphibian,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
animal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
warthog,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
whale,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
wombat,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
worm,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Compute the ppmi matrix

In [ ]:
def pmi_matrix(df, positive=True):
    col_totals = df.sum(axis=0)
    total = col_totals.sum()
    row_totals = df.sum(axis=1)
    expected = np.outer(row_totals, col_totals) / total
    df = df / expected
    with np.errstate(divide='ignore'):
        df = np.log(df)
    df[np.isinf(df)] = 0.0
    if positive:
        df[df < 0] = 0.0
    return df

In [ ]:
ppmi_matrix_df = pmi_matrix(datafr, positive=True)

In [ ]:
ppmi_matrix_df

,algae,alligator,amount,amphibian,animal,ant,anteater,aphid,aquatic,baboon,...,turtle,variety,vegetable,vegetation,walrus,warthog,whale,wombat,worm,zebra
algae,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
alligator,0.0,0.000000,0.0,2.809403,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
amount,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.809403,0.0,0.0,0.0,0.0,0.0
amphibian,0.0,2.809403,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
animal,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
warthog,0.0,0.000000,0.0,0.000000,0.0,0.0,2.809403,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
whale,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
wombat,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
worm,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


Get the top 5 couples of keywords for the first question

In [ ]:
# Flatten the upper triangular part of the PPMI matrix in order to avoid duplicates
upper_triangular = np.triu(ppmi_matrix_df.values, k=1).flatten()

# Get the indices of the top 5 highest values
top_indices = np.argsort(upper_triangular)[-5:]

# Convert the 1D indices to 2D indices
top_indices_2d = np.unravel_index(top_indices, ppmi_matrix_df.shape)

# Get the corresponding words and values
top_words_values = []
for row, col in zip(top_indices_2d[0], top_indices_2d[1]):
    word1, word2 = ppmi_matrix_df.columns[col], ppmi_matrix_df.index[row]
    if (word1, word2) not in top_words_values and (word2, word1) not in top_words_values:
        top_words_values.append((word1, word2, ppmi_matrix_df.iloc[row, col]))


print("Top 5 highest values without repetitions:\n")
for word1, word2, value in top_words_values:
    print(f"{word1} {word2} {value}")

Top 5 highest values without repetitions:

quokka make 3.7256934272366524
elephant carnivore 3.859224819861175
quokka berry 3.859224819861175
wombat quokka 4.013375499688434
tiger carnivore 4.013375499688434


# Sentence embedding similarity

In [ ]:
# model providing sentence embeddings
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# model = model.cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/452 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

## Cosine similarity between answers pairs

In [ ]:
sentence_embeddings = model.encode(df['answer'], convert_to_tensor=True)

similarity_matrix = util.pytorch_cos_sim(sentence_embeddings, sentence_embeddings) # squared matrix

In [ ]:
similarity_values = [] # list of tuples made up in this way: (i, j, cosine_similarity_between_sentence_i_and_sentence_j)

# iteration over the upper triangular part of the similarity matrix in order to avoid duplicates (i.e to avoid the values in the main diagonal)
for i in tqdm(range(similarity_matrix.shape[0]-1)):
  for j in range(i+1, similarity_matrix.shape[0]):
    similarity_values.append((i, j, similarity_matrix[i][j].item()))


# optimized
# similarity_values = [(i, j, similarity_matrix[i, j].item()) for i in tqdm(range(similarity_matrix.shape[0]-1)) for j in range(i+1, similarity_matrix.shape[0])]

100%|██████████| 4091/4091 [00:53<00:00, 76.56it/s] 


In [ ]:
similarity_values

[(0, 1, 0.8159724473953247),
 (0, 2, 0.42194780707359314),
 (0, 3, 0.769945502281189),
 (0, 4, 0.6531356573104858),
 (0, 5, 0.45154932141304016),
 (0, 6, 0.7101806402206421),
 (0, 7, 0.5540552139282227),
 (0, 8, 0.5232322216033936),
 (0, 9, 0.781162679195404),
 (0, 10, 0.5220910906791687),
 (0, 11, 0.35648006200790405),
 (0, 12, 0.43281954526901245),
 (0, 13, 0.6941969394683838),
 (0, 14, 0.4792565703392029),
 (0, 15, 0.4399411976337433),
 (0, 16, 0.34800583124160767),
 (0, 17, 0.4772951602935791),
 (0, 18, 0.288474440574646),
 (0, 19, 0.5545313358306885),
 (0, 20, 0.3473818600177765),
 (0, 21, 0.4520722031593323),
 (0, 22, 0.5761790871620178),
 (0, 23, 0.6059361696243286),
 (0, 24, 0.6084239482879639),
 (0, 25, 0.5681227445602417),
 (0, 26, 0.6270241141319275),
 (0, 27, 0.623162031173706),
 (0, 28, 0.4889494478702545),
 (0, 29, 0.5465688705444336),
 (0, 30, 0.44342362880706787),
 (0, 31, 0.3329295516014099),
 (0, 32, 0.5293653607368469),
 (0, 33, 0.6468932032585144),
 (0, 34, 0.570487

In [ ]:
sorted_similarity_values = sorted(similarity_values, key=lambda x: x[2], reverse=True) # descending sorted similarity values

In [ ]:
sorted_similarity_values

[(3519, 3695, 0.9905060529708862),
 (3475, 3695, 0.987808883190155),
 (3519, 3739, 0.987473726272583),
 (3475, 3519, 0.9865509271621704),
 (1404, 1492, 0.9865257143974304),
 (1406, 1494, 0.9859697222709656),
 (1397, 1485, 0.9859378337860107),
 (1374, 1462, 0.9852805137634277),
 (3695, 3739, 0.9841188192367554),
 (1385, 1473, 0.9838272929191589),
 (3475, 3739, 0.9820057153701782),
 (1368, 1456, 0.9818832874298096),
 (1396, 1484, 0.981865644454956),
 (1367, 1455, 0.9810245633125305),
 (1374, 1418, 0.9797676801681519),
 (1389, 1477, 0.9787786602973938),
 (2668, 2712, 0.9784848690032959),
 (1162, 1294, 0.9775537252426147),
 (263, 527, 0.9772189259529114),
 (1391, 1479, 0.9768625497817993),
 (1160, 1292, 0.9755764007568359),
 (1349, 1393, 0.9747862815856934),
 (3739, 3783, 0.9744819402694702),
 (1412, 1456, 0.9742339253425598),
 (3563, 3739, 0.9741615056991577),
 (1250, 1294, 0.9739032983779907),
 (1305, 1349, 0.9739013314247131),
 (1206, 1294, 0.9738840460777283),
 (3519, 3563, 0.973851621

## Getting the five highest similarities with the corresponding answers

In [ ]:
for i in range(5):
  print('\nanswer: ', df['answer'][sorted_similarity_values[i][0]])
  print('\nanswer: ', df['answer'][sorted_similarity_values[i][1]])
  print('\nsimilarity: ', sorted_similarity_values[i][2])
  print('\n=====================================================')


answer:  The use of anteaters in circuses is not common, and it is generally discouraged due to ethical considerations and the specialized needs of these animals. 

answer:  The use of warthogs in circuses is not common, and it is generally discouraged due to ethical considerations and the specialized needs of these animals. 

similarity:  0.9905060529708862


answer:  The use of armadillos in circuses is not common, and it is generally discouraged due to ethical considerations and the specialized needs of these animals. 

answer:  The use of warthogs in circuses is not common, and it is generally discouraged due to ethical considerations and the specialized needs of these animals. 

similarity:  0.987808883190155


answer:  The use of anteaters in circuses is not common, and it is generally discouraged due to ethical considerations and the specialized needs of these animals. 

answer:  The use of meerkats in circuses is not common, and it is generally discouraged due to ethical consi

## Getting the top 10 words correlated starting from cosine similarity

In [ ]:
len_words = len(df['word'].unique())
cluster = pd.DataFrame(np.zeros((len_words, len_words)), columns=df['word'].unique(), index=df['word'].unique())

# Filter the pairs to only include those with similarity >= 0.9
filtered_list = [(idx1, idx2, score) for idx1, idx2, score in tqdm(sorted_similarity_values) if score >= 0.85]

for idx_1, idx_2, sim_score in filtered_list:
    if(idx_1 == idx_2):
        continue
    word_1 = df['word'][idx_1]
    word_2 = df['word'][idx_2]
    cluster.loc[word_1][word_2] += 1

top_10_values = cluster.unstack().nlargest(10)
for (word_1, word_2), value in top_10_values.items():
    print(f"{word_1} - {word_2} | Value: {value}")

100%|██████████| 8370186/8370186 [00:09<00:00, 860380.34it/s] 


alligator - crocodile | Value: 37.0
emu - ostrich | Value: 33.0
falcon - eagle | Value: 31.0
falcon - hawk | Value: 31.0
parrot - pigeon | Value: 30.0
goat - sheep | Value: 30.0
owl - eagle | Value: 29.0
rat - mouse | Value: 26.0
owl - hawk | Value: 25.0
parrot - peacock | Value: 25.0


# Selection of the best keywords using word2vec word similarity + my own idea

My own idea:
- compute the embeddings of the possible keywords of an answer by using word2vec
- compute the cosine similarity between all these embeddings
- if there is only one keyword, take it. Otherwise, select the keyword with the highest similarity resulting from a sum of all the similarities it is involved (i.e. it should be the keyword that better summarizes the answer)

In [ ]:
!pip install gensim

In [ ]:
from gensim.models import Word2Vec

In [ ]:
df

,word,question,answer,prep_answer,keyword
0,lion,Is he a carnivore or herbivore?,Lion is a carnivore.,[carnivore],carnivore
1,lion,Is it a vertebrate or invertebrate?,Lion is a vertebrate.,[vertebrate],vertebrate
2,lion,Has it ever been domesticated?,Lions have been trained to some extent in capt...,"[lion, train, extent, captivity, domesticate, ...",domesticate
3,lion,Is it a mammal?,"Yes, the lion is a mammal.","[yes, lion, mammal]",yes
4,lion,Does it have fur?,"Yes, lions have fur.","[yes, lion, fur]",yes
...,...,...,...,...,...
4087,quokka,What are its dimensions?,Quokkas are small marsupials with a body lengt...,"[small, marsupial, body, length, cm, inch, tai...",inch
4088,quokka,Does the animal have moist or scaly skin?,"Quokkas have fur-covered skin, and their skin ...","[fur, cover, skin, skin, moist]",skin
4089,quokka,Can the animal echolocate?,Quokkas do not echolocate. They rely on visual...,"[echolocate, rely, visual, vocal, communication]",echolocate
4090,quokka,Can the animal metamorphose?,"No, quokkas do not undergo metamorphosis. They...","[no, undergo, metamorphosis, give, birth, live...",no


Modified version of the preprocess_text function in order to avoid the concepts in the possible keywords

In [ ]:
def wv_preprocess_text(sentence, word, stopwords, lemmatize=True):
  doc = nlp(sentence)
  sentence_tokens = []
  for token in doc:

    if token.pos_ not in ['NOUN', 'ADJ', 'VERB'] and token.text.lower() not in {'yes', 'no'}:
      continue

    token_text = token.lemma_ if lemmatize else token.text
    token_text = token_text.lower()

    if token_text in stopwords or token_text == word: # here we do the new check
      continue
    sentence_tokens.append(token_text)
  return sentence_tokens

In [ ]:
wv_prep_answs = []

i = 0
for sentence in tqdm(df['answer']):
  wv_prep_answs.append(wv_preprocess_text(sentence, df['word'][i], stop_words, lemmatize=True))
  i+=1

100%|██████████| 4092/4092 [01:01<00:00, 66.85it/s] 


In [ ]:
wv_prep_answs

[['carnivore'],
 ['vertebrate'],
 ['train', 'extent', 'captivity', 'domesticate', 'dog', 'cat'],
 ['yes', 'mammal'],
 ['yes', 'fur'],
 ['find', 'grassland', 'savanna', 'open', 'woodland'],
 ['carnivore', 'eat', 'meat', 'hunt', 'group'],
 ['leg'],
 ['no', 'give', 'birth', 'live', 'young'],
 ['predator'],
 ['leg', 'no', 'wing', 'fin'],
 ['find', 'small', 'population'],
 ['wild', 'dangerous', 'human', 'close', 'encounter'],
 ['eat', 'human'],
 ['yes', 'know', 'powerful', 'agile', 'jump'],
 ['short', 'stand', 'human', 'body', 'length'],
 ['live',
  'social',
  'group',
  'call',
  'pride',
  'consist',
  'relate',
  'female',
  'offspring'],
 ['suitable', 'legal', 'keep', 'pet', 'wild', 'nature'],
 ['migrate',
  'long',
  'distance',
  'movement',
  'influence',
  'factor',
  'food',
  'availability',
  'water',
  'source'],
 ['intelligent', 'predator', 'consider', 'intelligent', 'animal'],
 ['classify', 'vulnerable', 'population', 'decrease', 'certain', 'region'],
 ['crepuscular', 'mean',

In [ ]:
for i in range (len(df)):
  df['prep_answer'][i] = wv_prep_answs[i]

Now in the dataframe we have the new preprocessed answers

In [ ]:
df

,word,question,answer,prep_answer,keyword
0,lion,Is he a carnivore or herbivore?,Lion is a carnivore.,[carnivore],carnivore
1,lion,Is it a vertebrate or invertebrate?,Lion is a vertebrate.,[vertebrate],vertebrate
2,lion,Has it ever been domesticated?,Lions have been trained to some extent in capt...,"[train, extent, captivity, domesticate, dog, cat]",domesticate
3,lion,Is it a mammal?,"Yes, the lion is a mammal.","[yes, mammal]",yes
4,lion,Does it have fur?,"Yes, lions have fur.","[yes, fur]",yes
...,...,...,...,...,...
4087,quokka,What are its dimensions?,Quokkas are small marsupials with a body lengt...,"[small, marsupial, body, length, cm, inch, tai...",inch
4088,quokka,Does the animal have moist or scaly skin?,"Quokkas have fur-covered skin, and their skin ...","[fur, cover, skin, skin, moist]",skin
4089,quokka,Can the animal echolocate?,Quokkas do not echolocate. They rely on visual...,"[echolocate, rely, visual, vocal, communication]",echolocate
4090,quokka,Can the animal metamorphose?,"No, quokkas do not undergo metamorphosis. They...","[no, undergo, metamorphosis, give, birth, live...",no


Word2Vec model

In [ ]:
model = Word2Vec(sentences=prep_answs, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
def get_word_vector(word, model):
    try:
        return model.wv[word]
    except KeyError:
        return None

In [ ]:
keywords_vectors = [] # list of vectors, each one containing the vector representation of the keyword of that answer

"""
keywords_vectors[0] = [vector_representation_of_keyword_1, vector_representation_of_keyword_2, ...]
keywords_vectors[1] = [vector_representation_of_keyword_1, vector_representation_of_keyword_2, ...]
.
.
.
keywords_vectors[4091] = [vector_representation_of_keyword_1, vector_representation_of_keyword_2, ...]
"""


for elem in df['prep_answer']:
  keywords_vector = []

  for keyword in elem:
    keywords_vector.append([keyword, get_word_vector(keyword, model)])

  keywords_vectors.append(keywords_vector)

In [ ]:
len(keywords_vectors) # len(df) is right

4092

In [ ]:
len(keywords_vectors[2]) # to check that all is ok so far

6

Here we select the best keywords. in case there is only one, it is taken. Otherwise, the similarity matrix is computed and the best keyword is the one that is involved in the highest similarity. In this way, we get the keyword that better summarizes the answer.

In [ ]:
selected_keywords = [] # list of selected keywords

k = 0
for elem in tqdm(keywords_vectors): # elem[0][0] is the keyword, elem[0][1] is the corresponding vector
  similarity_matrix = util.pytorch_cos_sim(elem[0][1], elem[0][1])

  if len(similarity_matrix) == 1: # this means that there is only one keyword
    selected_keywords.append(elem[0][0])

    k += 1

  else:
    similarity_values = {}

    # initialize each value in the dictionary equal to zero
    for i in tqdm(range(similarity_matrix.shape[0]-1)):
      for j in range(i+1, similarity_matrix.shape[0]):
        similarity_values[elem[0][1][i]] = 0
        similarity_values[elem[0][1][j]] = 0

    # iteration over the upper triangular part of the similarity matrix in order to avoid duplicates
    for i in tqdm(range(similarity_matrix.shape[0]-1)):
      for j in range(i+1, similarity_matrix.shape[0]):
        similarity_values[elem[0][1][i]] += similarity_matrix[i][j].item()
        similarity_values[elem[0][1][j]] += similarity_matrix[i][j].item()

    # here we take the best keyword
    best_keyword = max(similarity_values, key=similarity_values.get)

    selected_keywords.append(best_keyword)

    k += 1

100%|██████████| 4092/4092 [00:00<00:00, 8556.83it/s]


In [ ]:
len(selected_keywords) # len(df) is right

4092

In [ ]:
selected_keywords

['carnivore',
 'vertebrate',
 'train',
 'yes',
 'yes',
 'find',
 'carnivore',
 'leg',
 'no',
 'predator',
 'leg',
 'find',
 'wild',
 'eat',
 'yes',
 'short',
 'live',
 'suitable',
 'migrate',
 'intelligent',
 'classify',
 'crepuscular',
 'no',
 'yes',
 'yes',
 'yes',
 'no',
 'no',
 'mane',
 'mammal',
 'yes',
 'communicate',
 'no',
 'no',
 'no',
 'subject',
 'live',
 'no',
 'run',
 'adult',
 'fur',
 'no',
 'no',
 'find',
 'carnivore',
 'vertebrate',
 'domesticate',
 'yes',
 'yes',
 'inhabit',
 'carnivore',
 'leg',
 'no',
 'predator',
 'leg',
 'find',
 'yes',
 'eat',
 'yes',
 'tall',
 'solitary',
 'suitable',
 'migrate',
 'intelligent',
 'yes',
 'crepuscular',
 'no',
 'yes',
 'yes',
 'yes',
 'no',
 'no',
 'distinctive',
 'mammal',
 'solitary',
 'communicate',
 'no',
 'aquatic',
 'no',
 'subject',
 'live',
 'no',
 'run',
 'adult',
 'fur',
 'no',
 'no',
 'find',
 'herbivore',
 'vertebrate',
 'yes',
 'yes',
 'sparse',
 'inhabit',
 'herbivore',
 'leg',
 'no',
 'predator',
 'leg',
 'find',
 '

In [ ]:
df['keyword'] = selected_keywords

In [ ]:
wv_df = df.drop(columns=['answer', 'prep_answer'])

In [ ]:
wv_df

,word,question,keyword
0,lion,Is he a carnivore or herbivore?,carnivore
1,lion,Is it a vertebrate or invertebrate?,vertebrate
2,lion,Has it ever been domesticated?,train
3,lion,Is it a mammal?,yes
4,lion,Does it have fur?,yes
...,...,...,...
4087,quokka,What are its dimensions?,small
4088,quokka,Does the animal have moist or scaly skin?,fur
4089,quokka,Can the animal echolocate?,echolocate
4090,quokka,Can the animal metamorphose?,no


# TF-IDF
It is used to quantify the importance of a word in a document relative to a corpus

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
copy_df = df

In [ ]:
copy_df

,word,question,answer,prep_answer,keyword
0,lion,Is he a carnivore or herbivore?,Lion is a carnivore.,[carnivore],carnivore
1,lion,Is it a vertebrate or invertebrate?,Lion is a vertebrate.,[vertebrate],vertebrate
2,lion,Has it ever been domesticated?,Lions have been trained to some extent in capt...,"[train, extent, captivity, domesticate, dog, cat]",train
3,lion,Is it a mammal?,"Yes, the lion is a mammal.","[yes, mammal]",yes
4,lion,Does it have fur?,"Yes, lions have fur.","[yes, fur]",yes
...,...,...,...,...,...
4087,quokka,What are its dimensions?,Quokkas are small marsupials with a body lengt...,"[small, marsupial, body, length, cm, inch, tai...",small
4088,quokka,Does the animal have moist or scaly skin?,"Quokkas have fur-covered skin, and their skin ...","[fur, cover, skin, skin, moist]",fur
4089,quokka,Can the animal echolocate?,Quokkas do not echolocate. They rely on visual...,"[echolocate, rely, visual, vocal, communication]",echolocate
4090,quokka,Can the animal metamorphose?,"No, quokkas do not undergo metamorphosis. They...","[no, undergo, metamorphosis, give, birth, live...",no


In [ ]:
# successivly we will need the preprocessed answers as a list of string. So, let'd do it

str_list = []

for elem in list(copy_df["prep_answer"]):
  temp_str = ""
  for it in elem:
    temp_str = temp_str + " " + it + " "
  str_list.append(temp_str.strip())

In [ ]:
str_list

['carnivore',
 'vertebrate',
 'train  extent  captivity  domesticate  dog  cat',
 'yes  mammal',
 'yes  fur',
 'find  grassland  savanna  open  woodland',
 'carnivore  eat  meat  hunt  group',
 'leg',
 'no  give  birth  live  young',
 'predator',
 'leg  no  wing  fin',
 'find  small  population',
 'wild  dangerous  human  close  encounter',
 'eat  human',
 'yes  know  powerful  agile  jump',
 'short  stand  human  body  length',
 'live  social  group  call  pride  consist  relate  female  offspring',
 'suitable  legal  keep  pet  wild  nature',
 'migrate  long  distance  movement  influence  factor  food  availability  water  source',
 'intelligent  predator  consider  intelligent  animal',
 'classify  vulnerable  population  decrease  certain  region',
 'crepuscular  mean  active  dawn  dusk',
 'no  hibernate',
 'yes  sharp  tooth',
 'yes  sharp  claw',
 'yes  belong  family',
 'no  amphibian',
 'no  reptile',
 'mane  male  distinguishable  feature',
 'mammal  warm  blood',
 'yes  hun

In [ ]:
# Create the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
# Get the TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(str_list)

In [ ]:
terms = tfidf_vectorizer.get_feature_names_out()

In [ ]:
# Convert TF-IDF matrix into a list for better view
dense_tfidf_matrix = tfidf_matrix.todense()

In [ ]:
df_tfidf = pd.DataFrame(dense_tfidf_matrix, columns=terms)

In [ ]:
df_tfidf

,ability,absent,abundant,abuse,accident,accidental,accustomed,achieve,acrobatic,act,...,world,worm,wrinkle,wrist,year,yes,young,zeal,zone,zoo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.694513,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.629845,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
4088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
4089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.418019,0.0,0.0,0.0


Let's remove the columns corresponding to the words in the clean file. In this way, we ensure that they cannot be the keywords

In [ ]:
i = 0

while i < len(copy_df):
  # print(copy_df['word'][i])
  try:
    df_tfidf = df_tfidf.drop(columns=[copy_df['word'][i]])
  except:
    print('column not found or already removed from the dataframe')
  i += num_questions

column not found or already removed from the dataframe
column not found or already removed from the dataframe
column not found or already removed from the dataframe
column not found or already removed from the dataframe
column not found or already removed from the dataframe
column not found or already removed from the dataframe
column not found or already removed from the dataframe
column not found or already removed from the dataframe
column not found or already removed from the dataframe
column not found or already removed from the dataframe
column not found or already removed from the dataframe
column not found or already removed from the dataframe
column not found or already removed from the dataframe
column not found or already removed from the dataframe
column not found or already removed from the dataframe
column not found or already removed from the dataframe
column not found or already removed from the dataframe
column not found or already removed from the dataframe
column not

In [ ]:
df_tfidf

,ability,absent,abundant,abuse,accident,accidental,accustomed,achieve,acrobatic,act,...,world,worm,wrinkle,wrist,year,yes,young,zeal,zone,zoo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.694513,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.629845,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
4088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
4089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
4090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.418019,0.0,0.0,0.0


Now we take the keywords with the highest value for each line

In [ ]:
tf_idf_keywords = df_tfidf.idxmax(axis=1).tolist()

In [ ]:
len(tf_idf_keywords) # len(df) is right

4092

These are the obtained keywords

In [ ]:
tf_idf_keywords

['carnivore',
 'vertebrate',
 'extent',
 'mammal',
 'fur',
 'woodland',
 'carnivore',
 'leg',
 'young',
 'predator',
 'fin',
 'population',
 'encounter',
 'eat',
 'agile',
 'stand',
 'pride',
 'legal',
 'availability',
 'intelligent',
 'decrease',
 'dawn',
 'hibernate',
 'tooth',
 'claw',
 'belong',
 'amphibian',
 'reptile',
 'mane',
 'blood',
 'target',
 'marking',
 'scale',
 'aquatic',
 'arctic',
 'subject',
 'nearby',
 'fly',
 'burst',
 'weigh',
 'moist',
 'echolocate',
 'metamorphosis',
 'circus',
 'carnivore',
 'vertebrate',
 'individual',
 'mammal',
 'fur',
 'mangrove',
 'deer',
 'leg',
 'young',
 'predator',
 'fin',
 'country',
 'dangerous',
 'eat',
 'jumper',
 'massive',
 'solitary',
 'legal',
 'territory',
 'intelligent',
 'poaching',
 'dawn',
 'hibernate',
 'tooth',
 'claw',
 'belong',
 'amphibian',
 'reptile',
 'recognizable',
 'blood',
 'hunter',
 'marking',
 'scale',
 'good',
 'arctic',
 'subject',
 'city',
 'fly',
 'burst',
 'weigh',
 'moist',
 'echolocate',
 'metamorphos

In [ ]:
copy_df['keyword'] = tf_idf_keywords

In [ ]:
tf_df = copy_df.drop(columns=['answer', 'prep_answer'])

In [ ]:
tf_df

,word,question,keyword
0,lion,Is he a carnivore or herbivore?,carnivore
1,lion,Is it a vertebrate or invertebrate?,vertebrate
2,lion,Has it ever been domesticated?,extent
3,lion,Is it a mammal?,mammal
4,lion,Does it have fur?,fur
...,...,...,...
4087,quokka,What are its dimensions?,cm
4088,quokka,Does the animal have moist or scaly skin?,skin
4089,quokka,Can the animal echolocate?,vocal
4090,quokka,Can the animal metamorphose?,young
